In [137]:
def preprocess_text(text):
    """Preprocess text by keeping all characters as literals"""
    # Convert escape sequences to literal characters
    processed = text.encode('unicode_escape').decode('utf-8')
    return processed

In [138]:
def caesar_cipher(text, alphabet, shift, mode='encrypt', case_strategy='strict', ignore_foreign=False):
    """
    Encrypts or decrypts text using the Caesar cipher.
    
    Args:
        ignore_foreign (bool): If True, foreign characters not in alphabet are ignored
    """
    result = ""
    
    # Create case-specific alphabets if needed
    if case_strategy in ['maintain', 'ignore']:
        lower_alphabet = alphabet.lower()
        upper_alphabet = alphabet.upper()
    
    for char in text:
        if case_strategy == 'strict':
            if char in alphabet:
                index = alphabet.index(char)
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                result += alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
        elif case_strategy == 'maintain':
            if char.lower() in lower_alphabet:
                index = lower_alphabet.index(char.lower())
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                if char.isupper():
                    result += upper_alphabet[new_index]
                else:
                    result += lower_alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
        elif case_strategy == 'ignore':
            char_lower = char.lower()
            if char_lower in lower_alphabet:
                index = lower_alphabet.index(char_lower)
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                result += lower_alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
    return result

In [139]:
def encode_key(key, alphabet):
    """Encodes the key to a string using the given alphabet (base conversion)."""
    base = len(alphabet)
    if key == 0:
        return alphabet[0]
    
    digits = []
    while key:
        digits.append(alphabet[key % base])
        key //= base
    return "".join(digits[::-1])

def decode_key(encoded_key, alphabet):
    """Decodes the key from a string to an integer."""
    base = len(alphabet)
    key = 0
    for char in encoded_key:
        key = key * base + alphabet.index(char)
    return key

In [140]:
def encrypt_with_metadata(text, alphabet, shift, case_strategy='strict', ignore_foreign=False):
    """Encrypts text and includes metadata in output"""
    processed_text = preprocess_text(text)
    encoded_key = encode_key(shift, alphabet)
    encrypted = caesar_cipher(processed_text, alphabet, shift, 'encrypt', case_strategy, ignore_foreign)
    
    # Format: encoded_key|case_strategy|ignore_foreign|encrypted_text
    return f"{encoded_key}|{case_strategy}|{1 if ignore_foreign else 0}|{encrypted}"

def decrypt_with_metadata(encrypted_message, alphabet):
    """Decrypts text by extracting metadata from message"""
    try:
        # Split metadata
        encoded_key, case_strategy, ignore_foreign, encrypted_text = encrypted_message.split('|', 3)
        
        # Convert metadata
        shift = decode_key(encoded_key, alphabet)
        ignore_foreign = bool(int(ignore_foreign))
        
        # Decrypt using extracted metadata
        return caesar_cipher(encrypted_text, alphabet, shift, 'decrypt', case_strategy, ignore_foreign)
    except ValueError:
        raise ValueError("Invalid encrypted message format")

In [141]:
def encrypt_text(text, alphabet, encoded_shift, case_strategy='strict', ignore_foreign=False):
    """Encrypts text using Caesar cipher with encoded key."""
    text = preprocess_text(text)
    shift = decode_key(encoded_shift, alphabet)
    return caesar_cipher(text, alphabet, shift, mode='encrypt', 
                        case_strategy=case_strategy, 
                        ignore_foreign=ignore_foreign)

def decrypt_text(encrypted_text, alphabet, encoded_shift, case_strategy='strict', ignore_foreign=False):
    """Decrypts text using Caesar cipher with encoded key."""
    shift = decode_key(encoded_shift, alphabet)
    return caesar_cipher(encrypted_text, alphabet, shift, mode='decrypt',
                        case_strategy=case_strategy,
                        ignore_foreign=ignore_foreign)


In [142]:
def shift_alphabet(alphabet, shift):
    return alphabet[shift:] + alphabet[:shift]

In [143]:
# Test implementation
alphabet = "`,.pyfgcrl/=\\aoeuidhtns-;qjkxbmwvz"
text = "Hello, World!"
shift = 7

# Test cases
test_cases = [
    ('strict', False, "Hello, World!"),
    ('maintain', True, "Hello,World"),
    ('ignore', False, "hello, world!"),
    ('ignore', True, "hello,world")
]

print(f"Shifted alphabet: {shift_alphabet(alphabet, shift)}")    
for case_strategy, ignore_foreign, expected_text in test_cases:
    print(f"\nTest {case_strategy} (ignore_foreign={ignore_foreign})")
    
    # Encrypt with metadata
    encrypted = encrypt_with_metadata(text, alphabet, shift, case_strategy, ignore_foreign)
    print(f"Encrypted message: {encrypted}")
    
    # Decrypt with extracted metadata
    decrypted = decrypt_with_metadata(encrypted, alphabet)
    print(f"Decrypted: {decrypted}")
    
    # Verify
    assert decrypted == expected_text, "Decryption failed!"

Shifted alphabet: crl/=\aoeuidhtns-;qjkxbmwvz`,.pyfg

Test strict (ignore_foreign=False)
Encrypted message: c|strict|0|Hsuunr Wneuq!
Decrypted: Hello, World!

Test maintain (ignore_foreign=True)
Encrypted message: c|maintain|1|JsuunrYneuq
Decrypted: Hello,World

Test ignore (ignore_foreign=False)
Encrypted message: c|ignore|0|jsuunr yneuq!
Decrypted: hello, world!

Test ignore (ignore_foreign=True)
Encrypted message: c|ignore|1|jsuunryneuq
Decrypted: hello,world
